# Read Physio Data

It would be convenient to have a function which allows to simply read the data of a certain time range of the measurements with an IMU.

In [1]:
import os
import numpy as np
import pandas as pd

In [ ]:
os.getcwd()

In [ ]:
in_file = 'Subject_01/subject01.csv'

In [ ]:
data = pd.read_csv(in_file, skiprows=0, sep=',')

In [ ]:
data.head()

In [ ]:
num_steps = np.shape(data.values)[0]

sampling_rate = 256 # [Hz]

duration = num_steps/sampling_rate # [s]
duration/60

In [4]:
# 02:22.1 02:37.1
start_min = 2
start_sec = 22.1

stop_min = 2
stop_sec = 37.1

In [ ]:
signals = ['Acc', 'Gyr', 'Mag']
start_time = start_min*60 + start_sec # [s]
stop_time = stop_min*60 + stop_sec # [s]

start_index = round(start_time * sampling_rate)
stop_index = round(stop_time * sampling_rate)

if start_index >= stop_index:
    start_index='NaN'

try:
    data_dict = {}
    for signal in signals:
        data_dict[signal] = data.filter(regex=signal+'*').values[start_index:stop_index]
except:
    raise Exception('Error at selecting data from given time range.')

In [ ]:
stop_index

In [ ]:
data_dict['Acc']

In [2]:
def get_sensor_data(in_file, signals=['Acc','Gyr','Mag'], sampling_rate=256, start_time=None, stop_time=None, skip_rows=0, sep=','):
    '''
    Function to read sensor data from a file, in order to return data from selected sensors and time range.
    
    Inputs
    ------
    in_file: directory and file name of data (e.g. 'Subject_01/subject01.csv')
    
    signals: list of sensor signal abbreviations (have to be equal to the first letters of the data column names!)
    
    sampling_rate: sampling rate of the measured signals in Hz
    
    start_time: start time for selecting data in sec (if None --> start from beginning)
    
    stop_time: stop time for selecting data in sec (if None --> until end of data)
    
    skip_rows: number of rows to skip for pandas read_csv() function
    
    sep: seperator for pandas read_csv() function
    
    
    Returns
    -------
    Dictionary with selected data and time array [s]
    '''
    
    data = pd.read_csv(in_file, skiprows=skip_rows, sep=sep)
    
    num_steps = np.shape(data.values)[0] # total number of data points
    
    if start_time is None:
        start_index = 0
    else:
        start_index = round(start_time * sampling_rate)
        
    if stop_time is None:
        stop_index = num_steps
    else:
        stop_index = round(stop_time * sampling_rate)
        
    if start_index < 0 or stop_index > num_steps or start_index >= stop_index:
        print('Error at selecting data from given time range.')
        return 0
        
    data_dict = {}
    for signal in signals:
        data_dict[signal] = data.filter(regex=signal+'*').values[start_index:stop_index]
        
    data_dict['time'] = np.arange(num_steps)[start_index:stop_index] / sampling_rate
    
    return data_dict

In [6]:
# 02:22.1 02:37.1
start_min = 2
start_sec = 22.1

stop_min = 2
stop_sec = 37.1

start_time = start_min*60 + start_sec # [s]
stop_time = stop_min*60 + stop_sec # [s]

In [7]:
import functionsMasterProjectMeinhart as fmpm
sensor_data = fmpm.get_sensor_data(in_file='Subject_01/subject01.csv', start_time=start_time, stop_time=stop_time)

In [8]:
np.shape(sensor_data['time'])

(3840,)

In [9]:
sensor_data['time']

array([142.1015625 , 142.10546875, 142.109375  , ..., 157.08984375,
       157.09375   , 157.09765625])

In [19]:
startMin = int(sensor_data['time'][0] / 60)
startSec = sensor_data['time'][0] - startMin*60

stopMin = int(sensor_data['time'][-1] / 60)
stopSec = sensor_data['time'][-1] - stopMin*60

print('Start time: ', startMin, 'min', '%.3f'%(startSec), 'sec')

print('Stop time:  ', stopMin, 'min', '%.3f'%(stopSec), 'sec')

Start time:  2 min 22.102 sec
Stop time:   2 min 37.098 sec


In [20]:
np.shape(sensor_data['Acc'])

(3840, 3)